In [2]:
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn
import json
import pandas as pd

In [36]:
d = {'piece': [], 'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
df = pd.DataFrame(data=d)

In [85]:
# [len([j['href'] for j in i]) for i in df['bulletin_url']]

In [75]:
# df = dfcopy.copy()
# dfcopy = df.copy()

In [113]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210707&to=20220501&output=json'
# url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20211231&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [156]:
final_urls[0]

'https://web.archive.org/web/20210709115515/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights'

In [115]:
def getdate(piece):
    date = [i for i in piece if i != '\n']
    date = [i for i in date if len(i.text) < 18 ][0].text
    return "Released: " + date.rstrip()
    

In [116]:
def getpieces(url):
    # Open page for roundup
    req = rq.get(url).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')

    # Find the section elements from the page
    sections = soup.find_all('section')
    # Filter out ones without h3 elements (headlines change between h3 and h2)
    pieces = []
    for section in sections:
        if section.find_all('h3') != []:
            pieces.append(section)

    pieces = [i for i in pieces if i.find('h3').text not in ["Further information", "Latest COVID-19 headlines", "Explore by topic", "Also published recently"]]
    return pieces
len(getpieces(final_urls[0]))

3

In [157]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,ERR: URL FAILED,placeholder


In [118]:
getpieces(final_urls[0])

[<section>
 <h3 id="antibodies">COVID-19 antibodies continue to rise in line with vaccinations across the UK countries</h3><h4>7 July 2021</h4><p>An estimated 89.8% of the adult population in England, 91.8% in Wales, 87.2% in Northern Ireland and 84.7% in Scotland tested positive for COVID-19 antibodies in the week beginning 14 June 2021. </p><p>During the same week, estimated vaccination rates continued to increase in all four UK countries. </p><p>The presence of antibodies suggests a person previously had COVID-19 or has been vaccinated. </p><p>Antibody positivity increases with age, reflecting age prioritisation in vaccination programmes. Vaccination rates are lowest in younger age groups but are increasing. Across the four UK countries, the estimated percentage of adults aged 25 to 34 years who have received one or more doses of COVID-19 vaccination ranged from 75.7% to 79.6%. This has increased sharply since the end of May 2021. </p><h4>Across all four UK countries, there is a cle

In [61]:
# This is a function to format a single roundup entry 
def makeru(p):
        
    # Put in a list and remove line breaks
    p = [i for i in p if i not in ['\n']]
    p = [i for i in p if (i.text not in ['Analysis | Data', 'Download the data'])&('Embed this interactive' not in i.text)]
    
    p2 = []
    last = 'Not Notes'
    for n, i in enumerate(p):
        if (len(i.text) < 20) & (n < 3):
            pass
        else:
            if i.text == 'Notes':
                last = 'Notes'
            else:
                if last != 'Notes':
                    p2.append(i)
                last = 'Not Notes'
                
    
    # Remove all h4s and h5s which are the dates and chart titles, filter out elements that contain text suggesting its furniture
    p2 = [i for i in p2 if (i.name not in ['h2', 'h4', 'h5', 'summary', 'div']) and ('Embed code' not in i.text) and ('XLSX' not in i.text) and ('Explore the latest' not in i.text) and ('Read the latest' not in i.text) and ('Read the full bulletin' not in i.text) and ('Back to the top' not in i.text) and ('Download this chart' not in i.text)]
        
    # For each element in the section, if it is a list iteratre the list adding a dash to the listed text, if not a list then just get text from element
    p2 = [ [ " - " + i.text for i in q if i != '\n' ] if q.name == 'ul' else q.text for q in p2 ]

    string = ""
    
    for i in p2:
        if type(i) == list:
            for j in i:
                string = string + j + '\n'
        else:
            string = string + i + '\n'
        
    return string

# print(makeru(pieces[5]))

In [62]:
def getbullurl(piece):
    reldate = getdate(piece)
    
    loas = [i['href'] for i in piece.findAll('a') if ('articles/coronaviruscovid19latestinsights' not in i['href']) and ('ons.gov.uk' in i['href']) ]
    loas = ["http" + i.split("http")[-1]  for i in loas]
    
    links = []
    
    for e in loas:
        if e.split("/")[-1] == 'latest':
            
            req = rq.get(e).text
            soup = bs(req,'html.parser')
            prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']
            
            linkseg = 'null'
            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    
                    
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')
                    
                    if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                        break
                        
            if linkseg == 'null':
                
                if 'Coronavirus and the social impacts' in soup.find('h1').text:
                    prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

                elif 'Economic activity and social change' in soup.find('h1').text:
                    prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'
                
                for i in range(1, 20):
                    req = rq.get( prevrelurl + "?page=" + str(i) ).text
                    soup = bs(req,'html.parser')

                    if '404 - ' not in soup.find('title').text:
                        lis = soup.find("main").find("div", class_="print-links").find_all('li')

                        if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                            linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                            break
                            
            if linkseg == 'null':
                belletinurl = "ERR: NO PREVIOUS RELEASE FOR :: " + e + " DATE :: " + date
            else:
                bulletinurl = "https://www.ons.gov.uk" + linkseg
            
        elif '/releases/' not in e:
            bulletinurl = e
            
        else:    
            req = rq.get(e).text
            # parse roundup html using beautifulsoup and store in soup
            soup = bs(req,'html.parser')

            cols = soup.find_all('div', class_="col-wrap")

            pubs = [i for i in cols if 'Publications' in i.text]

            if len(pubs)>0:
                pub = pubs[0]
                pubind = cols.index(pub)
                linksect = cols[pubind+1]
                bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']

            else:
                bulletinurl = "ERR: NO PUBLICATION FOR :: " + e + " DATE :: " + date

        bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")
        
        links.append(bulletinurl)
                
    return links

In [63]:
def addpiecestodf(pieces, df):
    for j, piece in enumerate(pieces):
        if makeru(piece) not in [el for el in df['roundup']]:
            df = df.append({'piece': piece, 'roundup': makeru(piece), 'bulletin_url': 'placeholder', 'bulletin_mps': 'placeholder'}, ignore_index=True)
    return df


In [64]:
for i in range(1, len(final_urls)+1):
    print('i', i)
    print(final_urls[-i])
    pieces = getpieces(final_urls[-i])
    df = addpiecestodf(pieces, df)
df

i 1
https://web.archive.org/web/20220430170828/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
i 2
https://web.archive.org/web/20220430123803/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
i 3
https://web.archive.org/web/20220427020940/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
i 4
https://web.archive.org/web/20220426095424/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
i 5
https://web.archive.org/web/20220424120302/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights
i 6
https://web.archive.org/web/20220423194929/https://www.ons.gov.uk/peoplepopu

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/visualisations/dvc1501...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/visualisations/dvc1390...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/visualisations/dvc1449...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,placeholder,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,placeholder,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,placeholder,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,placeholder,placeholder


In [71]:
for i in df.index:
    if df.iloc[i]['bulletin_url'] == 'placeholder':
        try:
            if getbullurl(df.iloc[i]['piece']) == []:
                df.iloc[i]['bulletin_url'] = 'ERR: NO URL FOUND'
            else:
                df.iloc[i]['bulletin_url'] = getbullurl(df.iloc[i]['piece'])
        except:
            df.iloc[i]['bulletin_url'] = "ERR: URL FAILED"
            

In [73]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/visualisations/dvc1501...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/visualisations/dvc1390...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/visualisations/dvc1449...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,ERR: URL FAILED,placeholder


In [78]:
for i in df.index:
    if df['bulletin_url'][i] == []:
        df.iloc[i]['bulletin_url'] = 'ERR :: EMPTY BRACKETS'
    elif df['bulletin_url'][i][:3] != 'ERR':
        notin = ['/methodology/', '.xlsx']
        for j in notin:
            df['bulletin_url'][i] = [i for i in df['bulletin_url'][i] if j not in i]


In [79]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,ERR: URL FAILED,placeholder


In [92]:
def getmps(bulletinurl):
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            

    else:
        return "NO MAIN POINTS"
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

# print(getmps(df_wi_bul['bulletin_url'][0][0]))

In [84]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,ERR: URL FAILED,placeholder


In [87]:
df_wi_bul = df[df['bulletin_url'].apply(lambda x: x[:3] != 'ERR' )]
# df_no_bul = df[df['bulletin_url'].apply(lambda x: x[0] == 'E' )]
# df_multiurl = df[df['bulletin_url'].apply(lambda x: (len(x) > 1) & (x[0] != 'E') )]

In [96]:
df_wi_mps = df_wi_bul[df_wi_bul['bulletin_mps'].apply(lambda x: x != 'placeholder' )]

In [97]:
df_wi_bul

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, it is estimated that over 9 in 10 ..."
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,"There were 9,744 deaths in England and Wales r..."
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,Instances of reinfection with coronavirus (COV...
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In the weeks prior to 14 June 2021, the percen..."
...,...,...,...,...
554,"[\n, [Deaths from COVID-19 increased slightly ...",Deaths from COVID-19 increased slightly in Eng...,[https://www.ons.gov.uk/peoplepopulationandcom...,"There were 8,808 deaths in England and Wales r..."
555,"[\n, [COVID-19 antibodies continue to rise in ...",COVID-19 antibodies continue to rise in line w...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, it is estimated that around 9 in 1..."
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,Of all deaths registered in 2020 in England an...
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,Of all deaths registered in 2020 in England an...


In [122]:
df_wi_mps

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, it is estimated that over 9 in 10 ..."
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,"There were 9,744 deaths in England and Wales r..."
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,Instances of reinfection with coronavirus (COV...
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In the weeks prior to 14 June 2021, the percen..."
5,"[\n, [Economy, business and jobs], [The econom...",The economy grew by 2.3% in April 2021\nReal g...,[https://www.ons.gov.uk/economy/grossdomesticp...,UK gross domestic product (GDP) is estimated t...
...,...,...,...,...
554,"[\n, [Deaths from COVID-19 increased slightly ...",Deaths from COVID-19 increased slightly in Eng...,[https://www.ons.gov.uk/peoplepopulationandcom...,"There were 8,808 deaths in England and Wales r..."
555,"[\n, [COVID-19 antibodies continue to rise in ...",COVID-19 antibodies continue to rise in line w...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, it is estimated that around 9 in 1..."
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,Of all deaths registered in 2020 in England an...
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,Of all deaths registered in 2020 in England an...


In [301]:
[i for i in df_wi_mps['bulletin_mps'] if i == 'placeholder']

[]

In [93]:
for k, i in enumerate(df_wi_bul.index):
    if k > -1:
        for j in range(1, len(df_wi_bul['bulletin_url'][i]) + 1 ):
            url = df_wi_bul['bulletin_url'][i][-j]
            url = url.replace("http:/www", "https://www")
            mps = getmps(url)
            if mps != "NO MAIN POINTS":
                df_wi_bul['bulletin_mps'][i] = mps
                break

/Users/theojolliffe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [307]:
df_wi_mps.loc[4]

piece           [\n, [People in deprived areas less likely to ...
roundup         People in deprived areas less likely to have h...
bulletin_url    [https://www.ons.gov.uk/peoplepopulationandcom...
bulletin_mps    Across all ages, of those eligible, the follow...
Name: 4, dtype: object

In [140]:
import random

In [302]:
roundupsum_train = {}
roundupsum_train['data'] = []

roundupsum_val = {}
roundupsum_val['data'] = []

In [308]:
for i in df_wi_mps.index:
    if random.random() < 0.05:
        roundupsum_val['data'].append({
            'document': df_wi_mps.loc[i]['bulletin_mps'],
            'summary': df_wi_mps.loc[i]['roundup'],
            'id': i
        })
    else:
        roundupsum_train['data'].append({
            'document': df_wi_mps.loc[i]['bulletin_mps'],
            'summary': df_wi_mps.loc[i]['roundup'],
            'id': i
        })

In [154]:
with open('roundupsum_train.json', 'w') as outfile:
    json.dump(roundupsum_train, outfile)
    
with open('roundupsum_val.json', 'w') as outfile:
    json.dump(roundupsum_val, outfile)

In [155]:
! git add .
! git commit -m "first commit"
! git push -u origin main

[main 491b92c] first commit
 14 files changed, 14576 insertions(+), 3348 deletions(-)
 create mode 100644 .ipynb_checkpoints/CovSelenium-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/SeleniumRoundup-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/econ_high_rouge_len-checkpoint.csv
 create mode 100644 .ipynb_checkpoints/quotes-checkpoint.csv
 create mode 100644 CovSelenium.ipynb
 rewrite RUSCRAPE2.ipynb (94%)
 create mode 100644 SeleniumRoundup.ipynb
 create mode 100644 econ_high_rouge_len.csv
 create mode 100644 quotes.csv
 rewrite roundupsum_train.json (89%)
 rewrite roundupsum_val.json (98%)
Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 16 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (14/14), 137.96 KiB | 3.28 MiB/s, done.
Total 14 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 1 local object.
To https://github.com/theojolliffe/roundupdata.git
   7d20dc

In [315]:
df['source'] = 'COVID'
df

,piece,roundup,bulletin_url,bulletin_mps,source
0,"[\n, [10.1% of registered deaths in England an...",10.1% of registered deaths in England and Wale...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder,COVID
1,"[\n, [Infection rates decrease across the UK],...",Infection rates decrease across the UK\nThe pe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder,COVID
2,"[\n, [7 in 10 people in England have had COVID...",7 in 10 people in England have had COVID-19 \n...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder,COVID
3,"[\n, [Retail and recreation activity close to ...",Retail and recreation activity close to pre-co...,"[E, R, R, , :, :, , E, M, P, T, Y, , B, R, ...",placeholder,COVID
4,"[\n, [People in deprived areas less likely to ...",People in deprived areas less likely to have h...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder,COVID
...,...,...,...,...,...
551,"[\n, [Deaths involving COVID-19], [Up to 4 Sep...","Up to 4 September 2020, there were 52,376 deat...","[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder,COVID
552,"[\n, [UK labour market], [Our , [latest figure...",Our latest figures on the UK labour market hav...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder,COVID
553,"[\n, [GDP, July 2020], [Monthly gross domestic...",Monthly gross domestic product (GDP) grew by 6...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder,COVID
554,"[\n, [Mergers and acquisitions], [[Mergers and...",Mergers and acquisitions involving UK companie...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder,COVID


In [ ]:
final_urls

In [319]:
# This is a function to format a single roundup entry 
def makeruECON(p):
        
    # Put in a list and remove line breaks
    p = [i for i in p if i not in ['\n']]
    p = [i for i in p if (i.text not in ['Analysis | Data', 'Download the data'])&('Embed this interactive' not in i.text)]
    
    p2 = []
    last = 'Not Notes'
    for n, i in enumerate(p):
        if (len(i.text) < 20) & (n < 3):
            pass
        else:
            if i.text == 'Notes':
                last = 'Notes'
            else:
                if last != 'Notes':
                    p2.append(i)
                last = 'Not Notes'
                
    
    # Remove all h4s and h5s which are the dates and chart titles, filter out elements that contain text suggesting its furniture
    p2 = [i for i in p2 if (i.name not in ['h2', 'h4', 'h5', 'summary', 'div']) and ('Embed code' not in i.text) and ('XLSX' not in i.text) and ('Explore the latest' not in i.text) and ('Read the latest' not in i.text) and ('Read the full bulletin' not in i.text) and ('Back to the top' not in i.text) and ('Download this chart' not in i.text)]
        
    # For each element in the section, if it is a list iteratre the list adding a dash to the listed text, if not a list then just get text from element
    p2 = [ [ " - " + i.text for i in q if i != '\n' ] if q.name == 'ul' else q.text for q in p2 ]

    string = ""
    
    for i in p2:
        if type(i) == list:
            for j in i:
                string = string + j + '\n'
        else:
            string = string + i + '\n'
        
    return string

# print(makeru(pieces[5]))

In [454]:
url = final_urls[1]
url

'https://web.archive.org/web/20210126114928/https://www.ons.gov.uk/economy/economicoutputandproductivity/output/articles/ukeconomylatest/2021-01-25'

In [121]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [9 out of 10 adults tested positive for C...",9 out of 10 adults tested positive for COVID-1...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [UK COVID-19 deaths continue to rise], [3...",UK COVID-19 deaths continue to rise\nThe numbe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [How excess deaths have changed each mont...",How excess deaths have changed each month wher...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [COVID-19 reinfections are rare and milde...",COVID-19 reinfections are rare and milder than...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
4,"[\n, [Coronavirus (COVID-19) infections increa...",Coronavirus (COVID-19) infections increased in...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
...,...,...,...,...
556,"[\n, [North West and London had highest corona...",North West and London had highest coronavirus ...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
557,"[\n, [Deaths in England and Wales fall below f...",Deaths in England and Wales fall below five-ye...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
558,"[\n, [Almost 9 in 10 adults say they have had ...",Almost 9 in 10 adults say they have had their ...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
559,"[\n, [[\n, <div class=""pym-interactive pym-int...",This page provides an overview of the coronavi...,ERR: URL FAILED,placeholder


In [23]:
# def getpiecesEcon(url):
#     # Open page for roundup
#     req = rq.get(url).text
#     # parse roundup html using beautifulsoup and store in soup
#     soup = bs(req,'html.parser')
#     print(soup)
    
#     # Find the section elements from the page
#     sections = soup.find_all('section')
#     # Filter out ones without h3 elements (headlines change between h3 and h2)
#     pieces = []
#     for section in sections:
#         if section.find_all('h2') != []:
#             pieces.append(section)
    

#     pieces = [i for i in pieces if i.find('h2').text not in ["Latest COVID-19 headlines", "Explore by topic", "Also published recently"]]
#     return pieces

# getpiecesEcon(final_urls[1])

In [24]:
# def addpiecestodfEcon(pieces, df):
#     for j, piece in enumerate(pieces):
#         if makeruECON(piece) not in [el for el in df['roundup']]:
#             df = df.append({'piece': piece, 'roundup': makeru(piece), 'bulletin_url': 'placeholder', 'bulletin_mps': 'placeholder', 'source': 'ECONOMY'}, ignore_index=True)
#     return df


In [26]:
# for i in range(1, len(final_urls)+1):
#     print('i', i)
#     print(final_urls[-i])
#     pieces = getpiecesEcon(final_urls[-i])
#     df = addpiecestodfEcon(pieces, df)
# df

In [123]:
from rouge import Rouge 
rouge = Rouge()

In [125]:
df_wi_mps['rougef1'] = 'ph'

<ipython-input-125-8354e40957a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wi_mps['rougef1'] = 'ph'


In [126]:
for i in df_wi_mps.index:
    df_wi_mps['rougef1'][i] = rouge.get_scores(df_wi_mps.loc[i]['roundup'], df_wi_mps.loc[i]['bulletin_mps'])[0]['rouge-2']['f']

/Users/theojolliffe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [128]:
df_wi_mps['len_ratio'] = 'ph'

<ipython-input-128-c451c8096ebf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wi_mps['len_ratio'] = 'ph'


In [130]:
for i in df_wi_mps.index:
    df_wi_mps['len_ratio'][i] = len(df_wi_mps['bulletin_mps'][i]) / len(df_wi_mps['roundup'][i])

In [131]:
df_wi_mps=df_wi_mps.sort_values(by=['rougef1'])

In [132]:
df_wi_mps_highR = df_wi_mps[df_wi_mps['rougef1'] > 0.15]
# df_wi_mps_highR=df_wi_mps_highR.sort_values(by=['rougef1'])
df_wi_mps_lowR = df_wi_mps[df_wi_mps['rougef1'] < 0.15]
# df_wi_mps_lowR=df_wi_mps_lowR.sort_values(by=['rougef1'])

In [567]:
[i for i in df_wi_mps_highR['bulletin_mps']][1]

'The number of deaths registered in England and Wales in the week ending 20 November 2020 (Week 47) was 12,535; this was 281 more deaths than in Week 46.\nIn Week 47, the number of deaths registered was 20.8% above the five-year average (2,155 deaths higher).\nOf the deaths registered in Week 47, 2,697 mentioned "novel coronavirus (COVID-19)", accounting for 21.5% of all deaths in England and Wales; an increase of 231 deaths compared with Week 46.\nOf the 2,697 deaths involving COVID-19, 2,361 had this recorded as the underlying cause of death (87.5%).\nOf the 2,605 deaths that involved Influenza and Pneumonia, 280 had this recorded as the underlying cause of death (10.7%).\nThe number of deaths in hospitals, care homes and private homes was above the five-year average but deaths in other locations were below the five-year average in Week 47.\nIn England, the total number of deaths increased from 11,495 (Week 46) to 11,675 (Week 47); all English regions had a higher number of deaths th

In [566]:
[i for i in df_wi_mps_highR['roundup']][1]

'COVID-19 deaths rise for 11th week in a row, but at a slower rate\nThere were 2,697 deaths involving the coronavirus (COVID-19) in England and Wales in the week ending 20 November 2020, the highest weekly figure since the week ending 15 May 2020. \nThe latest increase in COVID-19 mortality was the 11th in succession, but the rise of 231 deaths compared with the previous week was the smallest in six weeks.\nBy region, the highest number of COVID-19 deaths was reported in the North West (629 deaths), followed by Yorkshire and The Humber (481). In total, there were 12,535 deaths across England and Wales in the week ending 20 November 2020, of which 21.5% involved COVID-19. \nTotal deaths in the latest week remained higher than normal for this time of year, with hospitals, care homes and private homes all recording deaths in excess of the five-year average. \n\nBased on area of usual residence. Geographical boundaries are based on the most up-to-date information available at the time of p

In [134]:
df_high_rouge_len = df_wi_mps_highR[df_wi_mps_highR['len_ratio'] > 1]

In [135]:
df_high_rouge_len

,piece,roundup,bulletin_url,bulletin_mps,rougef1,len_ratio
192,"[\n, [COVID-19 deaths rise for 11th week in a ...","COVID-19 deaths rise for 11th week in a row, b...",[https://www.ons.gov.uk/peoplepopulationandcom...,The number of deaths registered in England and...,0.154176,1.03324
511,"[\n, [COVID-19 antibodies among adults remain ...",COVID-19 antibodies among adults remain high \...,[https://blog.ons.gov.uk/2021/04/28/antibodies...,"In England, it is estimated that over 9 in 10 ...",0.15625,1.27348
174,"[\n, [Impact of the coronavirus pandemic on et...",Impact of the coronavirus pandemic on ethnic m...,[https://www.ons.gov.uk/peoplepopulationandcom...,Most ethnic groups in the UK experienced a wor...,0.156379,1.08108
521,"[\n, [COVID-19 antibodies continue to rise alo...",COVID-19 antibodies continue to rise along wit...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, it is estimated that over 9 in 10 ...",0.156522,1.06387
401,"[\n, [Young adults have highest coronavirus in...",Young adults have highest coronavirus infectio...,[https://www.ons.gov.uk/peoplepopulationandcom...,To ensure our latest estimates on coronavirus ...,0.158568,1.07426
...,...,...,...,...,...,...
351,"[\n, [Increase in lateral flow testing among v...",Increase in lateral flow testing among vaccina...,[https://www.ons.gov.uk/chartimage?uri=/people...,Data collected between 31 January and 5 Februa...,0.696356,1.0213
398,"[\n, [More than 96% of the population have COV...",More than 96% of the population have COVID-19 ...,[https://www.ons.gov.uk/peoplepopulationandcom...,The following statistics are the percentage of...,0.72327,1.22586
457,"[\n, [Many clinically extremely vulnerable peo...",Many clinically extremely vulnerable people st...,[https://www.ons.gov.uk/chartimage?uri=/people...,Most people previously considered to be clinic...,0.746377,1.00724
283,"[\n, [10.1% of registered deaths in England an...",10.1% of registered deaths in England and Wale...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In the week ending 15 April 2022 (Week 15), 9,...",0.751412,1.14599


In [138]:
df_high_rouge_lenE = pd.read_csv('./econ_high_rouge_len.csv')

In [139]:
df_high_rouge_lenE

,page,roundup,bulletin_url,bulletin_mps,rougef1,len_ratio
0,https://web.archive.org/web/20210517053715/htt...,UK online job adverts in catering and hospital...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 6 May 2021, the aggregate CHAPS...",0.156306,2.741425
1,https://web.archive.org/web/20210605155126/htt...,Number of online job adverts on 21 May 2021 hi...,https://www.ons.gov.uk/releases/economicactivi...,We are currently undertaking research to impro...,0.156667,1.977243
2,https://web.archive.org/web/20210126034402/htt...,Coronavirus hits young and poorly-paid furloug...,https://www.ons.gov.uk/peoplepopulationandcomm...,The labour market shocks associated with the c...,0.162162,1.390119
3,https://web.archive.org/web/20220407125827/htt...,UK spending and OpenTable seated diners increa...,https://www.ons.gov.uk/releases/economicactivi...,"Both UK spending on credit and debit cards, an...",0.170330,1.694196
4,https://web.archive.org/web/20210413063033/htt...,The proportion of businesses currently trading...,https://www.ons.gov.uk/businessindustryandtrad...,The percentage of businesses currently trading...,0.172185,1.116781
5,https://web.archive.org/web/20211215074538/htt...,Inflation at its highest rate in over a decade...,https://www.ons.gov.uk/releases/consumerpricei...,The Consumer Prices Index including owner occu...,0.173469,2.974138
6,https://web.archive.org/web/20210501142704/htt...,Monthly retail sales volumes rose above pre-pa...,https://www.ons.gov.uk/businessindustryandtrad...,Retail sales volumes continued to recover in M...,0.176656,1.976996
7,https://web.archive.org/web/20210812100801/htt...,"UK flights surpassed seven-day average of 3,00...",https://www.ons.gov.uk/economy/economicoutputa...,"In the week ending 8 August 2021, the seven-da...",0.177419,3.620447
8,https://web.archive.org/web/20210811194228/htt...,Average UK daily flights up 117% since May 202...,https://www.ons.gov.uk/economy/economicoutputa...,"In the week to 1 August 2021, the seven-day av...",0.178261,4.047820
9,https://web.archive.org/web/20210126034402/htt...,"Retail sales rose in December, but saw a recor...",https://www.ons.gov.uk/businessindustryandtrad...,"In December 2020, retail sales volumes increas...",0.181818,1.072785


In [147]:
roundupsum_train = {}
roundupsum_train['data'] = []

roundupsum_val = {}
roundupsum_val['data'] = []

In [150]:
for i in df_high_rouge_len.index:
    if random.random() < 0.05:
        roundupsum_val['data'].append({
            'document': df_high_rouge_len.loc[i]['bulletin_mps'],
            'summary': df_high_rouge_len.loc[i]['roundup'],
            'id': i
        })
    else:
        roundupsum_train['data'].append({
            'document': df_high_rouge_len.loc[i]['bulletin_mps'],
            'summary': df_high_rouge_len.loc[i]['roundup'],
            'id': i
        })

In [153]:
len(roundupsum_val['data'])

20

In [161]:
len(roundupsum_train['data'])

264